# Homework assignment 1
Name: Klaas Schuijtemaker

Student nr.: 11163119

Course: Information Retrieval

Date: 11 jan. 2016

## Class definitions
We start of by defining some classes and their functions.

The first class we need is something to represent a document or webpage:

In [1]:
# Document/ Webpage 
class Doc:
    # Input:
    #   rel: relevance of the document 
    def __init__(self, rel):
        self.rel = rel
        
    # representation
    def __repr__(self):
        return 'Doc(' + str(self.rel) + ')'

In [2]:
# Test the Doc -class
d = Doc(3)
print(d)
print('Relevance:', d.rel)

Doc(3)
Relevance: 3


Next we need a class to hold the documents returned by a query:

In [3]:
import math

# Query session: a list of documents
class QuerySession:
    doc_list = []
    click_list = []
    
    # Input:
    #   doc_list: list with Docs ordered by rank
    def __init__(self, doc_list):
        self.doc_list = doc_list
        
    # representation
    def __repr__(self):
        return 'QuerySession(' + str(self.doc_list) + ')'
    
    # Discounted Cumulative Gain
    #   Input:
    #     k: rank position
    #   Output: DCG at rank k
    def dcg_evaluation(self, k = 5):
        out = 0
        for i in range(1, k + 1):
            out += (math.pow(2, self.doc_list[i - 1].rel) - 1) / math.log(i + 1, 2)
        return out

    # Rank Biased Precision
    #   Input: 
    #     p: persistence parameter
    #   Output: RBP
    def rbp_evaluation(self, p = 0.8):
        out = 0
        for i in range(1, len(self.doc_list) + 1):
            out += self.doc_list[i - 1].rel * math.pow(p, i)
        out *= 1 - p
        return out
    
    # Expected Reciprocal Rank
    #   Input: 
    #     gmax: maximum relevance
    #   Output: ERR
    def err_evaluation(self, gmax = 4):
        out = 0
        p = 1
        for r in range(1, len(self.doc_list) + 1):
            ri = (math.pow(2, self.doc_list[r - 1].rel - 1)) / (math.pow(2, gmax))
            out += p * ri / r
            p *= 1 - ri
        return out

In [4]:
# Test the QuerySession -class
qs = QuerySession([Doc(3),Doc(1),Doc(4),Doc(2),Doc(5)])
print(qs)
print('First doc:', qs.doc_list[0], '\n')

# Test evaluations:
print('DCG evaluation:', qs.dcg_evaluation())
print('RBP evaluation:', qs.rbp_evaluation())
print('ERR evaluation:', qs.err_evaluation())

QuerySession([Doc(3), Doc(1), Doc(4), Doc(2), Doc(5)])
First doc: Doc(3) 

DCG evaluation: 28.415396452062424
RBP evaluation: 1.50912
ERR evaluation: 0.463134765625


Next a class to handle interleaving:

In [5]:
import random

# Pair: two QuerySessions.
class Pair:
    
    # Input:
    #   qs1: QuerySession 1
    #   qs2: QuerySession 2
    def __init__(self, qs1, qs2):
        self.qs1 = qs1
        self.qs2 = qs2
        
    # representation
    def __repr__(self):
        return 'Pair(' + str(self.qs1) + ', ' + str(self.qs2) + ')'
    
    # Team-Draft Interleaving
    #   Input:
    #     length: number of documents
    #   Output: [i_list, team_a, team_b]
    def team_draft_interleaving(self):
        i1 = 0
        i2 = 0
        team_a = []
        team_b = []
        i_list = []
        while i1 < len(self.qs1.doc_list) or i2 < len(self.qs2.doc_list):
            if len(team_a) < len(team_b) or (len(team_a) == len(team_b) and random.getrandbits(1) == 1):
                if self.qs1.doc_list[i1] not in i_list:
                    i_list.append(self.qs1.doc_list[i1])
                    team_a.append(self.qs1.doc_list[i1])
                i1 += 1
            else:
                if self.qs2.doc_list[i2] not in i_list:
                    i_list.append(self.qs2.doc_list[i2])
                    team_b.append(self.qs2.doc_list[i2])
                i2 += 1
        return {'interleaved': QuerySession(i_list), 'team_a': team_a, 'team_b': team_b}

In [6]:
# Test the Pair -class
qs1 = QuerySession([Doc(1),Doc(2),Doc(3),Doc(4)])
qs2 = QuerySession([Doc(4),Doc(3),Doc(2),Doc(1)])
pair = Pair(qs1, qs2)
print(pair, '\n')

# Test Team-Draft Interleaving:
inter = pair.team_draft_interleaving()
print('Team-Draft Interleaving:', inter['interleaved'])
print('Team A:', inter['team_a'])
print('Team B:', inter['team_b'])

Pair(QuerySession([Doc(1), Doc(2), Doc(3), Doc(4)]), QuerySession([Doc(4), Doc(3), Doc(2), Doc(1)])) 

Team-Draft Interleaving: QuerySession([Doc(4), Doc(1), Doc(3), Doc(2), Doc(2), Doc(3), Doc(1), Doc(4)])
Team A: [Doc(1), Doc(2), Doc(3), Doc(4)]
Team B: [Doc(4), Doc(3), Doc(2), Doc(1)]


Next we need a class handle many pairs:

In [7]:
# Pair-list: unordered list containing Pairs: [Pair5, Pair3, Pair2, ...]
class PairList(list):
 
    # representation
    def __repr__(self):
        return 'PairList([\n' + ',\n'.join(str(d) for d in self[0:2]) + ',\n...,\n' + ',\n'.join(str(d) for d in self[-3:-1]) + '\n])'

    # Split PairList into multiple PairList
    #   Input:
    #     no_groups: split into num of groups
    #     function: evaluation measure that is used to split
    #   Output: a list containing PairLists: [PairList3, PairList2, PairList1, ...]
    def split(self, no_groups, function):
        measure = []
        for i in range(len(self)):
            pair = self[i]
            measure.append(function(pair.qs2) - function(pair.qs1)) # 𝛥measure = measure_e - measure_p

        group_range = max(measure) / no_groups # 0 < 𝛥measure_group1 ≤ group_range * 1 ≤ 𝛥measure_group2 ≤ group_range * 2 ...
        group = [PairList([]) for i in range(no_groups)] # group = [[],[],[], ...]
        for i in range(len(self)):
            if measure[i] > 0:
                group_index = int(math.ceil(measure[i] / group_range) - 1) # group_index range from 0 to 10
                group[group_index].append(self[i])
        return group

## Step 1: Simulate Rankings of Relevance for E and P

The assignment said that the relavance point scale should range from 0 to 5. However, this range resulted in a PairList that was simely too big to handle. Therefore the point scale range was lowered and now ranges from 0 to 3. 

In [8]:
point_scale = 3      # relevance point scale, ranges from 0 to 3

p = e = []
for i1 in range(point_scale):
    for i2 in range(point_scale):
        for i3 in range(point_scale):
            for i4 in range(point_scale):
                for i5 in range(point_scale):
                    p.append(QuerySession([Doc(i1), Doc(i2), Doc(i3), Doc(i4), Doc(i5)]))

pairs = PairList([])
for i1 in range(len(p)):
    for i2 in range(len(e)):
        pairs.append(Pair(p[i1], e[i2])) # add all possible combinations for P with E

In [9]:
# Check some of the pairs
print(pairs)

PairList([
Pair(QuerySession([Doc(0), Doc(0), Doc(0), Doc(0), Doc(0)]), QuerySession([Doc(0), Doc(0), Doc(0), Doc(0), Doc(0)])),
Pair(QuerySession([Doc(0), Doc(0), Doc(0), Doc(0), Doc(0)]), QuerySession([Doc(0), Doc(0), Doc(0), Doc(0), Doc(1)])),
...,
Pair(QuerySession([Doc(2), Doc(2), Doc(2), Doc(2), Doc(2)]), QuerySession([Doc(2), Doc(2), Doc(2), Doc(2), Doc(0)])),
Pair(QuerySession([Doc(2), Doc(2), Doc(2), Doc(2), Doc(2)]), QuerySession([Doc(2), Doc(2), Doc(2), Doc(2), Doc(1)]))
])


## Step 2: Divide pairs into groups of 10

In [10]:
no_groups = 10 # number of groups

dcg_groups = pairs.split(no_groups, QuerySession.dcg_evaluation)
rbp_groups = pairs.split(no_groups, QuerySession.rbp_evaluation)
err_groups = pairs.split(no_groups, QuerySession.err_evaluation)

all_groups = [dcg_groups, rbp_groups, err_groups]

In [11]:
# Check the Discounted Cumulative Gain -group
print('dcg_group[0]:')
print(dcg_groups[0])

dcg_group[0]:
PairList([
Pair(QuerySession([Doc(0), Doc(0), Doc(0), Doc(0), Doc(0)]), QuerySession([Doc(0), Doc(0), Doc(0), Doc(0), Doc(1)])),
Pair(QuerySession([Doc(0), Doc(0), Doc(0), Doc(0), Doc(0)]), QuerySession([Doc(0), Doc(0), Doc(0), Doc(1), Doc(0)])),
...,
Pair(QuerySession([Doc(2), Doc(2), Doc(2), Doc(2), Doc(0)]), QuerySession([Doc(2), Doc(2), Doc(2), Doc(1), Doc(2)])),
Pair(QuerySession([Doc(2), Doc(2), Doc(2), Doc(2), Doc(0)]), QuerySession([Doc(2), Doc(2), Doc(2), Doc(2), Doc(1)]))
])


## Step 3: Implement Team-Draft Interleaving

Interleaving is already included in the class Pair. We now replace all pairs in all groups with the interleaved ranking.

In [12]:
# Replace pairs by interleaves
for groups in all_groups:
    for i, group in enumerate(groups):
        i_list = []
        for pair in group:
            i_list.append(pair.team_draft_interleaving())
        groups[i] = i_list

In [13]:
# There should be 10 Discounted Cumulative Gain -groups:
print('DCG num. of groups:', len(dcg_groups))

# Each group has a variable amount of interleaves:
print('DCG size of group[0]:', len(dcg_groups[0]))

# One interleaf has an interleaved-ranking, a team A and a team B:
print('DCG size of group[0][0]:', len(dcg_groups[0][0]))

DCG num. of groups: 10
DCG size of group[0]: 7839
DCG size of group[0][0]: 3


## Step 4: Simulate User Clicks

In [14]:
# Add support for nested dictonaries: {key1: {key2: value2}}
class NestedDict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value

First we define a function to read a Yandex Click Log File:

In [15]:
# Load a Yandex Click Log File:
#   Input:
#     file: Yandex Click Log File
#   Output: session_map[session_id][query_id] containing alls QuerySession in the Yandex file
def load_yandex_click_log(file):
    session_map = NestedDict()
    session_id = 0
    for line in file:
        cells = line.split('\t')
        
        new_session_id = int(cells[0])
        if session_id != new_session_id:
            session_id = new_session_id
            
        if cells[2] == 'Q':
            query_id = int(cells[3])
            if query_id in session_map[session_id]:
                session = session_map[session_id][query_id]
            else:
                session = QuerySession([int(c.strip()) for c in cells[5:]])
                session.click_list = [0] * len(session.doc_list)
                session_map[session_id][query_id] = session
        
        if cells[2] == 'C':
            doc_id = int(cells[3].strip())
            for s in session_map[session_id].values():
                if doc_id in s.doc_list: 
                    rank = s.doc_list.index(doc_id)
                    s.click_list[rank] += 1
                    break
    return session_map

Implement the click models:

In [16]:
# Random click model estimate the probability of a click
#   Input:
#     session_map[session_id][query_id]
#   Output: Rho, the probability of a click
def rcm_get_click_prob(session_map):
    click_count = 0
    doc_count = 0
    for query_map in session_map.values():
        for session_query in query_map.values():
            for clicks in session_query.click_list:
                click_count += clicks                  # Count the number of clicks
        doc_count += len(session_query.doc_list)       # Count the number of documents
    return click_count / doc_count                     # Rho = click_count / doc_count

In [17]:
# Simplified Dependent Click Model estimate the continuation probability
#   Input:
#     session_map[session_id][query_id]
#   Output: Lambda r, a list[rank] with the continuation probability at each rank
def sdcm_get_continuation_probs(session_map):
    click_list = [0] * 10
    click_not_last_list = [0] * 10
    for query_map in session_map.values():
        for session_query in query_map.values():
            click_ranks = [r for r, click in enumerate(session_query.click_list) if click]
            if len(click_ranks) > 0:
                click_list[click_ranks[-1]] += session_query.click_list[click_ranks[-1]]
                for rank in click_ranks[:-1]:
                    click_list[rank] += session_query.click_list[rank]
                    click_not_last_list[rank] += session_query.click_list[rank]
    return [a / b for a, b in zip(click_not_last_list, click_list)]

In [18]:
# Simplified Dependent Click model estimate the attraction probability
#   Input:
#     query_session
#   Output: Alpha r, a list[rank] with the attraction probability at each rank
def sdcm_get_attraction_probs(query_session):
    return [d.rel / 4 for d in query_session.doc_list]

Decide -stochastically- whether a document was clicked 

In [19]:
# Random click model, estimate whether a document is clicked
#   Input:
#     rcm_click_prob: the probability of a click
#   Output: True if this document was clicked, False otherwise
def bernoulli(click_prob):
    return random.random() < click_prob

In [20]:
# Simplified Dependent Click model, estimate whether a document is clicked
#   Input:
#     query_session: a query session containing a ranked list of documents
#     cont_probs: Lambda r, a list[rank] with the continuation probability at each rank
#     attr_probs: Alpha r, a list[rank] with the attraction probability at each rank
#   Output: A click list[rank]: [True, False, False]
def sdcm_was_clicked(query_session, cont_probs, attr_probs):
    exam = 1
    click_list = []
    for rank in range(len(query_session.doc_list)):
        cont = cont_probs[rank]
        attr = attr_probs[rank]
        if exam == 0:
            click = False
            exam = 0
        else:
            click = bernoulli(attr)
            exam = bernoulli(1 - click + cont * click)
        click_list.append(click)
    return click_list

Now that all functions are defined, it is time to load the Yandex-file and calulate some probabilities:

In [21]:
# load file
f = open('YandexRelPredChallenge.txt', 'r')
session_map = load_yandex_click_log(f)
f.close()

# calculate probabilities
rcm_click_prob = rcm_get_click_prob(session_map)
sdcm_cont_probs = sdcm_get_continuation_probs(session_map)

In [22]:
# Check probabilities
print('Random click probability:', round(rcm_click_prob, 2))
print('Continuation probabilities:', [round(c, 2) for c in sdcm_cont_probs], '\n')

# Test Simplified Dependent Click Model
qs = QuerySession([Doc(3),Doc(1),Doc(4),Doc(2),Doc(1)])
sdcm_attr_probs = sdcm_get_attraction_probs(qs)
sdcm_click_list = sdcm_was_clicked(qs, sdcm_cont_probs, sdcm_attr_probs)
print('Test', qs)
print('Test Attraction probabilities:', sdcm_attr_probs)
print('Test clicked-list:', sdcm_click_list)

Random click probability: 0.49
Continuation probabilities: [0.39, 0.57, 0.6, 0.59, 0.58, 0.57, 0.54, 0.46, 0.38, 0.0] 

Test QuerySession([Doc(3), Doc(1), Doc(4), Doc(2), Doc(1)])
Test Attraction probabilities: [0.75, 0.25, 1.0, 0.5, 0.25]
Test clicked-list: [True, False, False, False, False]


## Step 5: Simulate Interleaving Experiment

In [23]:
no_simulations = 50 # the number of simulations

try:
    for groups in all_groups:
        for group in groups:
            for inter in group:
                interleaved = inter['interleaved']
                team_a = inter['team_a']
                team_b = inter['team_b']

                # Random Click Model  x 50
                p_win_count = 0
                e_win_count = 0
                for i in range(no_simulations):
                    for doc in interleaved.doc_list:
                        if bernoulli(rcm_click_prob):
                            if doc in team_a:
                                p_win_count += 1
                            else:
                                if doc in team_b:
                                    e_win_count += 1
                                else:
                                    print('Error1: doc not part of A or B')
                                    raise LookupError
                
                # Save the proportion that E has won
                inter['rcm_e_prop'] = e_win_count / (p_win_count + e_win_count)
                
                # Simplified Dependent Click Model  x 50
                p_win_count = 0
                e_win_count = 0
                for i in range(no_simulations):
                    sdcm_attr_probs = sdcm_get_attraction_probs(interleaved)
                    click_list = sdcm_was_clicked(interleaved, sdcm_cont_probs, sdcm_attr_probs)
                    for rank, click in enumerate(click_list):
                        if click:
                            doc = interleaved.doc_list[rank]
                            if doc in team_a:
                                p_win_count += 1
                            else:
                                if doc in team_b:
                                    e_win_count += 1
                                else:
                                    print('Error2: doc not part of A or B')
                                    raise LookupError
                
                # Save the proportion that E has won
                inter['sdcm_e_prop'] = e_win_count / (p_win_count + e_win_count)
                
except LookupError:
    pass

In [24]:
# Check the proportion of E wins in the first interleaved ranking from the Discounted Cumulative Gain -group

# In the Random Click Model
print('Proportion of E wins in the very first RCM-test:', dcg_groups[0][0]['rcm_e_prop'])

# In the Simplified Dependent Click Model
print('Proportion of E wins in the very first SDCM-test:', dcg_groups[0][0]['sdcm_e_prop'])

Proportion of E wins in the very first RCM-test: 0.524
Proportion of E wins in the very first SDCM-test: 1.0


## Step 6: Compute Sample Size 

In [25]:
z_alpha = 1.645 # 5%
z_beta = 1.282 # 10%
p0 = 0.5

for groups in all_groups:
    for group in groups:
        for inter in group:
            # Compute Random Click Model sample size
            p = inter['rcm_e_prop']
            delta = p - p0
            if delta == 0:
                inter['rcm_sample_size'] = -1
            else:
                inter['rcm_sample_size'] = math.pow((z_alpha * math.sqrt(p0 * (1 - p0)) + z_beta * math.sqrt(p * (1 - p))) / abs(delta), 2)
                #inter['rcm_sample_size'] += 1 / delta # continuity correction
                
            # Compute Simplified Dependent Click Model sample size
            p = inter['sdcm_e_prop']
            delta = p - p0
            if delta == 0:
                inter['sdcm_sample_size'] = -1
            else:
                inter['sdcm_sample_size'] = math.pow((z_alpha * math.sqrt(p0 * (1 - p0)) + z_beta * math.sqrt(p * (1 - p))) / abs(delta), 2)
                #inter['sdcm_sample_size'] += 1 / delta # continuity correction

Report the sample sizes

In [26]:
def get_repr_quantile(array, q):
    return str(round(array[round((len(array) - 1) * q)], 2))

def print_quantiles(name, array):
    array.sort()
    print(name, 'sample size quantiles:')
    print('min:\t',    get_repr_quantile(array, 0.00))
    print('q5:\t',     get_repr_quantile(array, 0.05))
    print('median:\t', get_repr_quantile(array, 0.50))
    print('q95:\t',    get_repr_quantile(array, 0.95))
    print('max:\t',    get_repr_quantile(array, 1.00))
    print('')

In [27]:
# report measure sample sizes
measure_names = ['Discounted Cumulative Gain', 'Rank Biased Precision', 'Expected Reciprocal Rank']
for i, groups in enumerate(all_groups):
    sample_sizes = []
    for group in groups:
        for inter in group:
            if inter['rcm_sample_size'] != -1:
                sample_sizes.append(inter['rcm_sample_size'])
            if inter['sdcm_sample_size'] != -1:
                sample_sizes.append(inter['sdcm_sample_size'])
    print_quantiles(measure_names[i], sample_sizes)
    
# report click model sample sizes
rcm_sample_sizes = []
sdcm_sample_sizes = []
for groups in all_groups:
    for group in groups:
        for inter in group:
            if inter['rcm_sample_size'] != -1:
                rcm_sample_sizes.append(inter['rcm_sample_size'])
            if inter['sdcm_sample_size'] != -1:
                sdcm_sample_sizes.append(inter['sdcm_sample_size'])
print_quantiles('Random Click Model', rcm_sample_sizes)
print_quantiles('Simplified Dependent Click Model', sdcm_sample_sizes)

# report group sample sizes
sample_sizes_array = [[] for i in range(10)]
for groups in all_groups:
    for i, group in enumerate(groups):
        sample_sizes = sample_sizes_array[i]
        for inter in group:
            if inter['rcm_sample_size'] != -1:
                sample_sizes.append(inter['rcm_sample_size'])
            if inter['sdcm_sample_size'] != -1:
                sample_sizes.append(inter['sdcm_sample_size'])
for i, sample_sizes in enumerate(sample_sizes_array):
    print_quantiles('Group ' + str(i + 1), sample_sizes)

Discounted Cumulative Gain sample size quantiles:
min:	 2.71
q5:	 24.11
median:	 1822.46
q95:	 131727.5
max:	 666885.7

Rank Biased Precision sample size quantiles:
min:	 2.71
q5:	 24.55
median:	 1828.72
q95:	 131727.5
max:	 745568.05

Expected Reciprocal Rank sample size quantiles:
min:	 2.71
q5:	 24.2
median:	 1851.16
q95:	 131727.5
max:	 676481.11

Random Click Model sample size quantiles:
min:	 165.95
q5:	 1032.89
median:	 8229.45
q95:	 489370.65
max:	 745568.05

Simplified Dependent Click Model sample size quantiles:
min:	 2.71
q5:	 15.71
median:	 162.97
q95:	 11724.92
max:	 101784.68

Group 1 sample size quantiles:
min:	 2.71
q5:	 125.78
median:	 2905.82
q95:	 133860.76
max:	 686145.06

Group 2 sample size quantiles:
min:	 2.71
q5:	 60.58
median:	 2087.88
q95:	 131727.5
max:	 745568.05

Group 3 sample size quantiles:
min:	 2.71
q5:	 34.02
median:	 1467.67
q95:	 131727.5
max:	 666885.7

Group 4 sample size quantiles:
min:	 2.71
q5:	 21.59
median:	 1032.89
q95:	 131727.5
max:	 6764

## Step 7: Analysis

### Conclusion
The entire test simulates two non-existing algorithms named E and P. E is an experimental algorithm that should replace P. Before P can be replaced, E must proof to be better. This is done tests. But how many tests are needed to proof E better? This entire simulation is about determine the sample size needed to proof that E is better than P.

In each test, algorithm E was deliberately given an advantage over algorithm P. The tests were divided over 10 groups. E had a small advantage in group 1 and E had a large advantage in group 10. After all tests the sample size, needed to proof that E better, was calculated. It turned out that in group 1, a ‘large’ median sample size of 2905 is needed. While in group 10, a ‘small’ median sample size of 10 is needed. This is as expected; when there isn’t much difference between algorithms, the amount of tests needed to proof one better than the other, must be ‘large’.

Two click models were tested, the random click model (RCM) and the simplified dependent click model (SDCM). Because the RCM is almost entirely random, it can’t create a big difference between E and P. The median sample size found for the RCM is 8229 which is ‘large’. The SDCM seems to create a big difference between E and P and has a median sample size of 163 which is ‘small’.

Three evaluation methods were tested, the Discounted Cumulative Gain (DCG), the Rank Biased Precision (RBP) and the Expected Reciprocal Rank (ERR). All three evaluation methods scored similar with a median sample size of +-1830 which is ‘average’. A good evaluation method is preferred to create big difference between algorithms so that the amount of tests needed is minimal.

### Drawback
This method is good for giving an approximate of the amount of tests needed. However, the difference between the minimum and the maximum sample size is always big. For example, in group 10, the median sample size of 10 is 'small'. But the maximum sample size is 514250 which is 'incredibly large'. This raises the question: are 10 tests really going to be sufficient?

### Alternative
Instead of determining the sample size, one could decide to start the test immediately and let it run until a statistical difference between the two algorithms have been found.
